# **fGraph 0 detection from eigenvalues**


<div id="import_libraries">
    <h2>Import Libraries</h2>
</div>


In [1]:
#!pip install pandas
#!pip install scikit-learn
#!pip install matplotlib
#!pip install xgboost

To import the libraries that will be used in this lab, execute the cells below. 


In [14]:
# Import the libraries we need to use in this lab
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ast
import time
# model trainin set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
#models 
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
#model evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score

## Load the dataset




In [15]:
# load the dataset
path= "fgraph_eigen_8.csv"
# read the input data
df = pd.read_csv(path)
df


,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,...,FGRAPH_EIGEN3,FGRAPH_EIGEN4,FGRAPH_EIGEN5,FGRAPH_EIGEN6,FGRAPH_EIGEN7,FGRAPH_EIGEN8,FGRAPH_EIGEN9,FGRAPH_EIGEN10,FGRAPH_EIGEN11,FGRAPH_EIGEN12
0,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-7.151720e-01,-4.142136e-01,1.935432,...,-2.000000,-2.000000,-1.667742,-7.320508e-01,-2.267388e-01,-1.332268e-15,2.732051e+00,2.828427,3.200059,4.0
1,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-7.151720e-01,-4.142136e-01,1.935432,...,-2.445281,-2.000000,-1.287292,-8.402523e-01,-3.974333e-01,-7.177127e-02,2.617707e+00,3.042355,3.197411,4.0
2,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-7.151720e-01,-4.142136e-01,1.935432,...,-2.000000,-2.000000,-1.538445,-1.236068e+00,-1.236068e+00,-2.968418e-01,2.878345e+00,3.236068,3.236068,4.0
3,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-7.151720e-01,-4.142136e-01,1.935432,...,-2.000000,-2.000000,-1.511407,-1.236068e+00,-5.364049e-01,-3.576331e-02,2.802365e+00,3.012160,3.236068,4.0
4,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-7.151720e-01,-4.142136e-01,1.935432,...,-2.000000,-1.791997,-1.591352,-1.358032e+00,-3.484352e-01,9.251150e-02,2.567288e+00,3.096807,3.245414,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704,0,-2.618034,-2.618034,-1.618034,-1.618034,-0.561553,-0.381966,-3.819660e-01,4.384472e-01,0.618034,...,-1.456850,-1.456850,-0.543150,-5.431497e-01,-1.372714e-16,9.969958e-16,1.188901e+00,1.188901,4.000000,4.0
2705,0,-2.900523,-2.000000,-2.000000,-1.581807,-0.798880,-0.486482,4.346060e-16,1.341751e-15,0.591077,...,-1.618034,-1.302776,-0.697224,-3.819660e-01,-7.036912e-16,9.482206e-16,6.180340e-01,2.302776,4.000000,4.0
2706,0,-2.618034,-2.230725,-1.874076,-1.618034,-0.482859,-0.381966,-1.519943e-16,1.519943e-16,0.486456,...,-2.000000,-1.414214,-0.585786,-1.599715e-15,-2.573573e-16,2.915845e-17,1.168644e-16,1.414214,4.000000,4.0
2707,0,-2.618034,-2.230725,-1.874076,-1.618034,-0.482859,-0.381966,-1.519943e-16,1.519943e-16,0.486456,...,-2.000000,-1.414214,-0.585786,-1.599715e-15,-2.573573e-16,2.915845e-17,1.168644e-16,1.414214,4.000000,4.0


In [16]:
# Let's round up the columns to 6 decimals.
# Take all non-integer coulumns
cols_to_round = df.columns.difference(['COEFFICIENT_DEN'])

# Round those columns to, say, 6 decimals
df[cols_to_round] = df[cols_to_round].round(6)
df

,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,...,FGRAPH_EIGEN3,FGRAPH_EIGEN4,FGRAPH_EIGEN5,FGRAPH_EIGEN6,FGRAPH_EIGEN7,FGRAPH_EIGEN8,FGRAPH_EIGEN9,FGRAPH_EIGEN10,FGRAPH_EIGEN11,FGRAPH_EIGEN12
0,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-0.715172,-0.414214,1.935432,...,-2.000000,-2.000000,-1.667742,-0.732051,-0.226739,-0.000000,2.732051,2.828427,3.200059,4.0
1,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-0.715172,-0.414214,1.935432,...,-2.445281,-2.000000,-1.287292,-0.840252,-0.397433,-0.071771,2.617707,3.042355,3.197411,4.0
2,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-0.715172,-0.414214,1.935432,...,-2.000000,-2.000000,-1.538445,-1.236068,-1.236068,-0.296842,2.878345,3.236068,3.236068,4.0
3,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-0.715172,-0.414214,1.935432,...,-2.000000,-2.000000,-1.511407,-1.236068,-0.536405,-0.035763,2.802365,3.012160,3.236068,4.0
4,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-0.715172,-0.414214,1.935432,...,-2.000000,-1.791997,-1.591352,-1.358032,-0.348435,0.092511,2.567288,3.096807,3.245414,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704,0,-2.618034,-2.618034,-1.618034,-1.618034,-0.561553,-0.381966,-0.381966,0.438447,0.618034,...,-1.456850,-1.456850,-0.543150,-0.543150,-0.000000,0.000000,1.188901,1.188901,4.000000,4.0
2705,0,-2.900523,-2.000000,-2.000000,-1.581807,-0.798880,-0.486482,0.000000,0.000000,0.591077,...,-1.618034,-1.302776,-0.697224,-0.381966,-0.000000,0.000000,0.618034,2.302776,4.000000,4.0
2706,0,-2.618034,-2.230725,-1.874076,-1.618034,-0.482859,-0.381966,-0.000000,0.000000,0.486456,...,-2.000000,-1.414214,-0.585786,-0.000000,-0.000000,0.000000,0.000000,1.414214,4.000000,4.0
2707,0,-2.618034,-2.230725,-1.874076,-1.618034,-0.482859,-0.381966,-0.000000,0.000000,0.486456,...,-2.000000,-1.414214,-0.585786,-0.000000,-0.000000,0.000000,0.000000,1.414214,4.000000,4.0


# ChatGpt clasifier computation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


# 1. Extract features and target
y = df["COEFFICIENT_DEN"]
X = df.drop(columns=["COEFFICIENT_DEN"])


In [ ]:
# 2. Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
# 3. Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Train a classifier (Logistic Regression here)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_scaled, y_train)

# 5. Predict and evaluate
y_pred = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8007380073800738
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.83      0.83       330
           1       0.74      0.76      0.75       212

    accuracy                           0.80       542
   macro avg       0.79      0.79      0.79       542
weighted avg       0.80      0.80      0.80       542



### most important feature

In [21]:
# 7. Get feature importance from Logistic Regression
feature_names = X.columns
coefficients = clf.coef_[0]  # For binary classification, it's a 1D array

# Create a DataFrame for sorting and readability
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
})

# Sort by absolute value of coefficients
feature_importance_sorted = feature_importance.sort_values(by='abs_coefficient', ascending=False)

# Display the most relevant feature
most_relevant = feature_importance_sorted.iloc[0:5]
print("Most relevant features:")
print(most_relevant)


Most relevant features:
           feature  coefficient  abs_coefficient
10     DEN_EIGEN11    -2.278267         2.278267
11     DEN_EIGEN12     1.524199         1.524199
22  FGRAPH_EIGEN11    -1.484623         1.484623
0       DEN_EIGEN1     1.160693         1.160693
21  FGRAPH_EIGEN10    -1.095030         1.095030


### Decision tree

In [22]:
# 1. (Optional) scale features – not needed for decision trees but harmless
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Train a Decision Tree classifier
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train_scaled, y_train)

# 3. Predict and evaluate
y_pred = tree_clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7472324723247232
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.80      0.79       330
           1       0.68      0.67      0.67       212

    accuracy                           0.75       542
   macro avg       0.73      0.73      0.73       542
weighted avg       0.75      0.75      0.75       542



In [23]:
# 4. Feature importances from the decision tree
importances = tree_clf.feature_importances_
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("Most important features:")
print(feature_importance.iloc[0:4])

Most important features:
           feature  importance
22  FGRAPH_EIGEN11    0.186502
10     DEN_EIGEN11    0.168651
21  FGRAPH_EIGEN10    0.141105
11     DEN_EIGEN12    0.057904


### Random forest

In [24]:
from sklearn.ensemble import RandomForestClassifier

# 1. Train the Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train_scaled, y_train)

# 2. Predict and evaluate
y_pred_rf = rf_clf.predict(X_test_scaled)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.8339483394833949
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.85      0.86       330
           1       0.77      0.82      0.79       212

    accuracy                           0.83       542
   macro avg       0.82      0.83      0.83       542
weighted avg       0.84      0.83      0.83       542



In [26]:
# 3. Feature importances from the random forest
importances = rf_clf.feature_importances_
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("Most important features:")
print(feature_importance.iloc[0:4])

Most important features:
           feature  importance
22  FGRAPH_EIGEN11    0.117148
10     DEN_EIGEN11    0.114370
21  FGRAPH_EIGEN10    0.107355
18   FGRAPH_EIGEN7    0.058450


# Can the model fit the data?

Let's use the 100% of our data to train the model. Does a solution exist? Let's try the logistic regression

In [31]:
# Prepare features and labels
X = df.drop(columns=["COEFFICIENT_DEN"])
y = df["COEFFICIENT_DEN"]

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train on full data
clf = LogisticRegression(max_iter=1000)
clf.fit(X_scaled, y)

# Evaluate on same data
y_pred = clf.predict(X_scaled)
print("Accuracy on full dataset:", accuracy_score(y, y_pred))


Accuracy on full dataset: 0.8324104835732743


In [32]:
from sklearn.svm import SVC

# Train SVM on full dataset
svm_clf = SVC(kernel='rbf')  # Try 'rbf' (default), 'poly', 'linear', etc.
svm_clf.fit(X_scaled, y)

# Evaluate on the same dataset
y_pred_svm = svm_clf.predict(X_scaled)
print("SVM accuracy on full dataset:", accuracy_score(y, y_pred_svm))


SVM accuracy on full dataset: 0.9121447028423773


Polynomial fit

In [33]:
# Train SVM on full dataset
svm_clf = SVC(kernel='poly')  # Try 'rbf' (default), 'poly', 'linear', etc.
svm_clf.fit(X_scaled, y)

# Evaluate on the same dataset
y_pred_svm = svm_clf.predict(X_scaled)
print("SVM accuracy on full dataset:", accuracy_score(y, y_pred_svm))

SVM accuracy on full dataset: 0.8600959763750461


## Random forest (almost) perfect score

We see that the random forest reach a prefect score. If we remove the denominator eigenvalues it reaces an almost perfect score

In [34]:
rf_clf = RandomForestClassifier(n_estimators=200, random_state=42)
rf_clf.fit(X_scaled, y)

y_pred_rf = rf_clf.predict(X_scaled)
print("Random Forest accuracy:", accuracy_score(y, y_pred_rf))

Random Forest accuracy: 1.0


In [37]:
importances = rf_clf.feature_importances_
feat_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
feat_importance = feat_importance.sort_values(by='importance', ascending=False)

print("Top features:\n", feat_importance.head())


Top features:
            feature  importance
22  FGRAPH_EIGEN11    0.113125
10     DEN_EIGEN11    0.111241
21  FGRAPH_EIGEN10    0.101027
18   FGRAPH_EIGEN7    0.059612
20   FGRAPH_EIGEN9    0.051620


In [40]:
# Prepare features and labels
X = df.drop(columns=["COEFFICIENT_DEN",'DEN_EIGEN1', 'DEN_EIGEN2', 'DEN_EIGEN3',
       'DEN_EIGEN4', 'DEN_EIGEN5', 'DEN_EIGEN6', 'DEN_EIGEN7', 'DEN_EIGEN8',
       'DEN_EIGEN9', 'DEN_EIGEN10', 'DEN_EIGEN11', 'DEN_EIGEN12'])
y = df["COEFFICIENT_DEN"]


# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [41]:
rf_clf = RandomForestClassifier(n_estimators=200, random_state=42)
rf_clf.fit(X_scaled, y)

y_pred_rf = rf_clf.predict(X_scaled)
print("Random Forest accuracy:", accuracy_score(y, y_pred_rf))

Random Forest accuracy: 0.9977851605758582


In [43]:
# 5. Find misclassified samples
misclassified = df[y != y_pred]

print(f"Number of misclassified samples: {len(misclassified)}")
print("Misclassified indices:")
print(misclassified.index.tolist())

Number of misclassified samples: 454
Misclassified indices:
[0, 5, 8, 9, 13, 18, 20, 27, 46, 68, 73, 75, 87, 90, 92, 96, 99, 106, 115, 118, 121, 125, 126, 128, 134, 135, 136, 137, 143, 144, 149, 159, 167, 191, 193, 200, 202, 208, 213, 219, 220, 223, 226, 243, 244, 245, 249, 250, 252, 255, 256, 261, 265, 284, 286, 291, 296, 297, 303, 304, 312, 319, 321, 328, 333, 337, 339, 341, 359, 363, 364, 365, 372, 384, 388, 412, 419, 420, 422, 430, 431, 432, 436, 437, 440, 451, 454, 461, 472, 476, 477, 486, 496, 497, 520, 521, 522, 532, 545, 563, 583, 601, 604, 605, 606, 610, 611, 622, 629, 630, 650, 669, 671, 673, 677, 680, 683, 684, 689, 696, 698, 700, 709, 729, 733, 735, 736, 739, 749, 754, 755, 756, 763, 773, 774, 777, 785, 786, 787, 788, 806, 818, 820, 822, 828, 834, 835, 838, 840, 841, 843, 845, 849, 852, 854, 856, 866, 871, 873, 874, 876, 883, 884, 885, 886, 887, 888, 889, 896, 900, 901, 902, 913, 927, 932, 934, 935, 936, 937, 938, 942, 957, 960, 980, 984, 989, 994, 997, 999, 1002, 1005, 100

In [44]:
len(misclassified)/len(df)

0.16758951642672573

In [42]:
importances = rf_clf.feature_importances_
feat_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
feat_importance = feat_importance.sort_values(by='importance', ascending=False)

print("Top features:\n", feat_importance.head())

Top features:
            feature  importance
10  FGRAPH_EIGEN11    0.161398
9   FGRAPH_EIGEN10    0.139988
8    FGRAPH_EIGEN9    0.124719
0    FGRAPH_EIGEN1    0.089089
6    FGRAPH_EIGEN7    0.087516
